# Ein Überblick

---
Lernziel

- Sie verstehen den Zusammenhang einer (logistischer) Regression und Neuronalen Netzwerken.
---

Im letzten Notebook werden wir uns noch ein letztes Mal mit verschieden Netzwerk-Architekturen auseinandersetzen und wie diese zusammen hängen.

Dafür werden wir noch einmal mit dem MNIST Datensatz arbeiten. Zunächst laden wir wieder die Daten und normalisieren diese. Wir kodieren die Target Variable auch wieder zu One-Hot Vektoren.

In [ ]:
import numpy as np
import scipy
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
import torch
from torch import nn
from torch import optim
def min_max(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))
def one_hot(x):
    """Die Labels der Bilder müssen noch in Vektoren von Länge 10 codiert werden"""
    dod = len(set(x)) # Checkt wie viele verschieden Ziffern es im Datennsatz gibt
    target = np.zeros([x.shape[0], dod]) # Eine Matrix aus Nullen wird erstellt
    for i in range(x.shape[0]): # Der for-loop setzt eine 1 in die Matrix abhängig davon welches Label das Bild hat
        target[i, x[i]] = 1

    return target

In [ ]:
train_data = np.genfromtxt('https://uni-muenster.sciebo.de/s/vVaMBQUQf5evomw/download', delimiter=',', skip_header =False) #genfromtxt liest .txt Datein, mit delimiter ="," können auch .csv (comma seperated values) Datein einglesen werden  
test_data = np.genfromtxt('https://uni-muenster.sciebo.de/s/1l80v7o5iLPgDEX/download', delimiter=',', skip_header =False) # hier lesen wir die Test Daten ein
 
train_labels=train_data[:,0].astype(int) 
train_images = train_data[:,1:]

test_labels=test_data[:,0].astype(int)
test_images = test_data[:,1:]

In [ ]:
train_targets=one_hot(train_labels)
test_targets = one_hot(test_labels)

In [ ]:
train_images = min_max(train_images)
test_images = min_max(test_images)

In [ ]:
plt.imshow(train_images[0].reshape([28, 28]), cmap="gray")
print("Correct Label: %s" % train_labels[0])

# Lineare Regression

Wir beginnen mit einer einfachen Regression. Eine lineare Regression kann auch als neuronales Netz dargestellt werden.

<img src="Img/summary/lin_reg.png" width ="450px">

Die Ausgabe setzt sich aus der gewichteten Summe der Pixelwerte zusammen. Das heißt, jedem Pixel wird ein Gewicht zugewiesen. *Das Neuron kann auch noch eine Bias haben, aber das wird nicht gezeigt*. 

Da wir nur ein Ausgangsneuron haben, können wir auch nur eine Vorhersage treffen. Das bedeutet, dass wir nur eine binäre Klassifizierung durchführen können. Zum Beispiel: Ist auf dem Bild eine Fünf zu sehen? JA oder NEIN.

Wir können diese lineare Regression auch in Python durchführen.
Dazu verwenden wir die `train_images` als Eingabe und die Spalte von `train_targets`. In diesem Fall ist es die fünfte Spalte `train_targets[:,5]`.

In [ ]:
linear_reg_model = LinearRegression()
linear_reg_model.fit(train_images, train_targets[:,5])

Wir können die Gewichte mit `linear_reg_model.coef_` ausgeben. Es gibt insgesamt 784 Gewichte, eines für jedes Pixel. 

In [ ]:
linear_reg_model.coef_[:5], linear_reg_model.coef_.shape

Um zu sehen, wie gut unser Modell funktioniert, können wir die Funktion `.predict()` verwenden, um den Wert für unseren Testdatensatz vorherzusagen. Denken Sie daran, dass wir nur Nullen oder Einsen vorhersagen wollen.

`1` = "Fünf"

`0` = "Keine Fünf"

In [ ]:
pred_y = linear_reg_model.predict(test_images)
pred_y

Diese Werte sind weder `0` noch `1`. Wir müssen diese erst noch runden.

In [ ]:
pred_y = np.round(pred_y)
pred_y

Jetzt können wir auch die Accuracy berrechen:

In [ ]:
np.mean(pred_y== test_targets[:,5])

`0,9456` ist nicht so schlecht. Aber bedenken Sie, dass nur etwa 10 % der Bilder eine `5` zeigen. Das bedeutet auch, dass 90% der Bilder keine `5` zeigen. Für diese 90 % müsste unser Modell eine  `0` vorhersagen, um richtig zu sein. Wenn das Modell einfach eine `0` für alle Bilder vorhersagt, hätte es eine Accuarcy von `0,90`. Unsere Accuracy ist also vielleicht weniger gut als ursprünglich angenommen.


Wir haben noch ein weiteres Problem. Schauen Sie sich einmal die Vorhersagen für `pred_y[1677]` oder `pred_y[1162]` an.

In [ ]:
pred_y[1677],pred_y[1162]

Diese Werte sind weder `1` noch `0`. Wie konnte das passieren?
Bei einer linearen Regression verwenden wir keine Aktivierungsfunktionen. Daher kann die Ausgabe einer linearen Regression unbegrenzt große oder kleien Werte annehmen. Wenn die Werte außerhalb von `[-1.5, 1.5]` liegen, werden sie nicht auf `0` oder `1` gerundet.

Das ist zunächst kein Problem, wir könnten diesen Werten auch manuell `0` oder `1` zuweisen. Aber das Problem bleibt im Prinzip bestehen: Wie erlauben dem Modell Werte vorherzusagen, die außerhalb des möglichen Bereichs liegen. 

Eine `sigmoid`-Funktion verhindert dies. Sie transformiert alle Werte so, dass sie immer zwischen `0` und `1` liegen. Wir können also einfach eine `sigmoid`-Funktion an die lineare Regression "hängen". Damit wäre das Problem gelöst. Und genau das passiert bei der logistischen Regression.

<img src="Img/summary/log_reg.png" width="450px">

Wir können dies auch in Python berechnen.

In [ ]:
log_reg_model = LogisticRegression(solver = 'lbfgs', max_iter=1000,  random_state=134)
log_reg_model.fit(train_images, train_targets[:,5])
log_reg_model.coef_[0,256:261], log_reg_model.coef_.shape

Wir erhalten wieder `784` Weights. Für jeden Pixel eins. Wir sehen auch, dass unsere Vorhersagen für den Testdatensatz jetzt schon gerundet werden.

In [ ]:
pred_y = log_reg_model.predict(test_images)
pred_y

Auch hier berechnen wir wieder die Accuracy.

In [ ]:
np.mean(pred_y == test_targets[:,5])

Mit Hilfe der logistischen Regression könnten wir also die Genauigkeit erhöhen. Bisher unterscheiden wir aber nur zwischen "Fünf" und "Keine Fünf". Wir wollen aber eigentlich jede Ziffer erkennen können. Auch das ist mit logistischer Regression möglich.

Das heißt, wir haben mehrere Outputnodes. Insgesamt 10, für jede Ziffer eine. 

<img src="Img/summary/log_reg_2.png" width="540px">

Die Funktion `softmax` wird nun verwendet. Im Gegensatz zur `sigmoid`-Funktion stellt die `softmax`-Funktion sicher, dass die Summe der Aktivierungen über die 10 Outputs immer genau `1` ist. Würden wir die `sigmoid` Funktion verwenden, könnte es passieren, dass ein Bild als eine Fünf und eine Eins erkannt wird. 


Für diese logistische Regression müssen wir nun die kompletten `train_labels` Matrix hinzufügen. 

In [ ]:
log_reg_model_alle = LogisticRegression(solver = 'lbfgs', max_iter=1000,  random_state=134)
log_reg_model_alle.fit(train_images, train_labels)
log_reg_model_alle.coef_[0,256:261], log_reg_model_alle.coef_.shape

Die Weightmatrix `log_reg_model_alle.coef_` hat nun die Größe `[10,784]`. Also pro Outputneuron 784 Weights.

Auch jetzt erhalten wir Vorhersagen, diesen enthält die vom Model erkannte Zahl. 

In [ ]:
pred_y = log_reg_model_alle.predict(test_images)
pred_y

Wir berechnen erneut die Accuracy:

In [ ]:
np.mean(pred_y==test_labels)

Das Modell kann 92,5 % der Ziffern richtig erkennen. Das ist natürlich schlechter als vorher, aber diesmal ist die Aufgabe viel komplexer, denn es geht nicht nur um eine Ziffer, sondern es müssen alle Zahlen richtig erkannt werden. 
Mit einer einfachen logistischen Regression können wir also eine relativ gute Accuracy erreichen. 

Wozu brauchen wir dann neuronale Netze? Diese können uns auch die letzten Prozentpunkte an Leistung bringen. Der Unterschied zwischen unserem aktuellen Modell und einem neuronalen Netz ist das Fehlen von Hidden Layers. 

<img src="Img/summary/nn1.png" width="450px">

Wir werden dies auch mit PyTorch nachbauen:

In [ ]:

train_images =torch.tensor(train_images, dtype = torch.float32)
test_images =torch.tensor(test_images, dtype = torch.float32)

train_labels =torch.tensor(train_labels, dtype = torch.long)
test_labels =torch.tensor(test_labels, dtype = torch.long)

In [ ]:
simple_nn = nn.Sequential(nn.Linear(784,10),nn.ReLU() ,nn.Linear(10,10))
loss_funktion = nn.CrossEntropyLoss()
updater = optim.Adam(simple_nn.parameters(), lr = 0.01)

In [ ]:
torch.manual_seed(1234)
for epoch in range(135):
    updater.zero_grad()
    output = simple_nn(train_images)
    loss = loss_funktion(output, train_labels)
    loss.backward()
    updater.step()
    
    

Der Code sollte Ihnen inzwischen vertraut sein. Wenn wir uns jedoch die Genauigkeit ansehen, sehen wir, dass das neuronale Netz eine Accuracy hat, die mit der Accuracy der logistischen Regression vergleichbar ist.

In [ ]:
pred_y=torch.argmax(simple_nn(test_images),1).detach().numpy()

In [ ]:
np.mean(pred_y==test_labels.numpy())

Dies kann mehrere Gründe haben. Im Prinzip müssen neuronale Netze nicht besser funktionieren als einfachere Modelle.
In diesem Fall liegt es aber wahrscheinlich an unserem Modell selbst. Wir können mehr oder größere Layers verwenden. Oder wir ändern den Optimizer oder die Lernrate.



# Übungsaufgabe


Das heutige Notebook ist kürzer als sonst, so dass Sie mehr Zeit für die Übung haben. 
In der heutigen Übungsaufgabe geht es darum, das Gelernte noch einmal auf den MNIST-Datensatz anzuwenden. 

Sie erhalten drei Datensätze (zufällig gemischt):

- Trainingsdaten: verwenden Sie zum Trainieren
- Testdaten: zur Bewertung des trainierten Netzwerks
- Externer Testdatensatz: nur Bilder, keine Beschriftung → Sie schicken mir die Vorhersagen für diesen Datensatz.

Der externe Datensatz hat keine Lösungen (zumindest keine, die Sie einsehen können). 
Bei der Übungsaufgabe geben Sie auch **ihre** Vorhersagen für den externen Datensatz ab.

Wir werden dann Ihre Vorhersagen mit den wahren Werten vergleichen. 
*Wer von Ihnen erstellt das beste Modell?*

Es wurde ein erstes Modell vorgegeben.
Von dort aus können Sie Ihr Netzwerk verbessern.

Es gibt mehrere Möglichkeiten, Ihr Netzwerk zu verbessern:
Hier sind ein paar Beispiele.

- Anpassung von Hyperparametern, z. B. Anzahl der Epochen, Stapelgröße, Lernrate oder Anzahl der versteckten Schichten
- Batchnorm und Dropout
- CNN
- Optimierer

Achten Sie darauf, dass Sie den Testdatensatz nicht zu stark anpassen. Auch das kann passieren.

Am Ende des Codes befindet sich eine Zelle, mit der Sie die Vorhersage für den Testdatensatz erstellen und speichern können. 
Diese wird im Ordner `data` als `my_prediction.csv` gespeichert.

Bitte reichen Sie sowohl Ihre Vorhersage als auch das Notebook ein.


# Daten 

Laden Sie zunächst alle Daten.

In [ ]:
import numpy as np
import scipy
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch import optim
from torch.utils import data
import pandas as pd

def min_max(x):
    return (x - 0.) / (255. - 0.)


In [ ]:
train_data = np.genfromtxt('https://uni-muenster.sciebo.de/s/vVaMBQUQf5evomw/download', delimiter=',', skip_header =False)
train_labels=train_data[:,0].astype(int) 
train_images = min_max(train_data[:,1:])
del train_data 

test_data = np.genfromtxt('https://uni-muenster.sciebo.de/s/1l80v7o5iLPgDEX/download', delimiter=',', skip_header =False)
test_labels=test_data[:,0].astype(int)
test_images = min_max(test_data[:,1:])
del test_data 

external_images=min_max(np.genfromtxt('https://uni-muenster.sciebo.de/s/0kAd13OWqx1FPZD/download', delimiter=',', skip_header =False))


In [ ]:
train_images =torch.tensor(train_images, dtype = torch.float32)
test_images =torch.tensor(test_images, dtype = torch.float32)

train_labels = torch.tensor(train_labels, dtype = torch.long)
test_labels = torch.tensor(test_labels, dtype = torch.long)

external_images = torch.tensor(external_images, dtype = torch.float32)

In [ ]:
train_data = data.TensorDataset(train_images, train_labels) # input sind unsere Tensors die einmal die Bilder und einmal die Labels beinhalten
loader = data.DataLoader(train_data, batch_size = 32)

##  Model

In [ ]:
simple_nn = nn.Sequential(nn.Linear(784,10),nn.ReLU() ,nn.Linear(10,10))
loss_funktion = nn.CrossEntropyLoss()
updater = optim.Adam(simple_nn.parameters(), lr = 0.0001)

In [ ]:
torch.manual_seed(1234)
for epoch in range(20):
    simple_nn.train()
    for images, labels in loader:
        updater.zero_grad()
        output = simple_nn(images)
        loss = loss_funktion(output, labels)
        loss.backward()
        updater.step()
    
    simple_nn.eval()
    # EVALUATE #
    # Train
    output = simple_nn(train_images)
    loss = loss_funktion(output, train_labels)
    prediction = torch.argmax(output,1).detach().numpy()
    acc  = np.mean(prediction == train_labels.detach().numpy()  )
    # Tets
    output = simple_nn(test_images)
    test_loss = loss_funktion(output, test_labels)
    prediction = torch.argmax(output,1).detach().numpy()
    test_acc  = np.mean(prediction == test_labels.detach().numpy()  )
    print(f"Epoch {epoch} | Trainings Loss: {loss:.3f} Training Acc: {acc:.3f} | Test Loss: {test_loss:.3f} Test Acc:  {test_acc:.3f}")
        

# Externe Daten 

In [ ]:
simple_nn.eval()
externe_pred = torch.argmax(simple_nn(external_images),1).detach().numpy()

Die nächste Zelle generiert eine `.csv` Datein mit Ihren Vorhersagen. Diese reichen Sie bitte mit dem Notebook ein.

In [ ]:
pd.DataFrame(externe_pred.reshape(10000,1)).to_csv("../data/meine_prediction.csv", index =False,header =False)